In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import geopandas as gpd
from pyproj import datadir
import pandas as pd

datadir.set_data_dir("/home/jupyter-daniela/.conda/envs/peru_environment/share/proj")


In [ ]:

filename_chl_daily = '/home/jupyter-daniela/suyana/peru_production/features/cmems_chl_daily_2015-2025_0N_90W_20S_70W_0.25deg.nc'
filename_calas = '/home/jupyter-daniela/suyana/peru_production/outputs/calas_grillado_semanal.csv'

chl_data = xr.open_dataset(filename_chl_daily)
chl_surface = chl_data.sel(time=slice("2021-05-01", "2021-05-31")).isel(depth=0)

semana = (((chl_surface.time.dt.dayofyear - 1)//7) + 1).rename('semana')
chl_may_2021 = (chl_surface
                .assign_coords(semana=('time', semana.data))
                .groupby('semana')
                .mean('time'))


In [ ]:
calas_data = pd.read_csv(filename_calas)
calas_may_2021 = calas_data[calas_data['temporada'].str.contains('2021') & calas_data['semana'].between(18, 21)].copy()

calas_may_2021['lat_bin'] = pd.to_numeric(calas_may_2021['lat_bin'], errors='coerce')
calas_may_2021['lon_bin'] = pd.to_numeric(calas_may_2021['lon_bin'], errors='coerce')

boundary = gpd.read_file("/home/jupyter-daniela/suyana/geometries/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 8))
axes = axes.flatten()

semanas = [18, 19, 20, 21]

for i, semana in enumerate(semanas):
    ax = axes[i]

    if semana in chl_may_2021.semana.values:
        boundary.plot(ax=ax, facecolor='none', edgecolor='black')

        chl_week = chl_may_2021.sel(semana=semana)

        chl_plot = ax.contourf(
            chl_week.longitude,
            chl_week.latitude,
            chl_week,
            levels=20,
            cmap='Greens',
            vmin=0, vmax=7,
            alpha=0.8
        )

    calas_week = calas_may_2021[calas_may_2021['semana'] == semana]
    calas_week = calas_week[calas_week['suma_pescado'] > 0]

    if not calas_week.empty:
        scatter = ax.scatter(
            calas_week['lon_bin'],
            calas_week['lat_bin'],
            c=calas_week['suma_pescado'],
            cmap='Reds',
            s=30,
            alpha=0.8,
            edgecolors='black',
            linewidth=0.3,
            vmin=0, vmax=500

        )

    ax.set_xlim([-82, -76])
    ax.set_ylim([-15, -6])
    ax.set_title(f'Semana {semana}', fontsize=10)
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')

fig.colorbar(chl_plot, ax=axes, orientation='vertical', fraction=0.025, pad=.03, label='Clorofila (mg/m³)', shrink=0.7)
plt.show()

In [ ]:
filename_chl_daily = '/home/jupyter-daniela/suyana/peru_production/features/cmems_chl_daily-anom_2015-2025_0N_90W_20S_70W_0.25deg.nc'

chl_data = xr.open_dataset(filename_chl_daily)
chl_surface = chl_data.sel(time=slice("2021-05-01", "2021-05-31")).isel(depth=0)

semana = (((chl_surface.time.dt.dayofyear - 1)//7) + 1).rename('semana')
chl_may_2021 = (chl_surface
                .assign_coords(semana=('time', semana.data))
                .groupby('semana')
                .mean('time'))

fig, axes = plt.subplots(1, 4, figsize=(18, 8))
axes = axes.flatten()

semanas = [18, 19, 20, 21]

for i, semana in enumerate(semanas):
    ax = axes[i]

    if semana in chl_may_2021.semana.values:
        boundary.plot(ax=ax, facecolor='none', edgecolor='black')

        chl_week = chl_may_2021.sel(semana=semana)

        chl_plot = ax.contourf(
            chl_week.longitude,
            chl_week.latitude,
            chl_week['chl'],
            levels=20,
            cmap='RdBu_r',
            vmin=-3, vmax=3,
            alpha=0.8
        )

    calas_week = calas_may_2021[calas_may_2021['semana'] == semana]
    calas_week = calas_week[calas_week['suma_pescado'] > 0]

    if not calas_week.empty:
        scatter = ax.scatter(
            calas_week['lon_bin'],
            calas_week['lat_bin'],
            c=calas_week['suma_pescado'],
            cmap='Reds',
            s=30,
            alpha=0.8,
            edgecolors='black',
            linewidth=0.3,
            vmin=0, vmax=500

        )

    ax.set_xlim([-82, -76])
    ax.set_ylim([-15, -6])
    ax.set_title(f'Semana {semana}', fontsize=10)
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')

fig.colorbar(chl_plot, ax=axes, orientation='vertical', fraction=0.025, pad=.03, label='Clorofila (mg/m³)', shrink=0.7)
plt.show()

## SST

In [ ]:
filename_sst_daily = '/home/jupyter-daniela/suyana/peru_production/features/sst_weekly_2015_2024.nc'

sst_data = xr.open_dataset(filename_sst_daily)

# Calcular climatología (promedio histórico por semana del año)
sst_data_with_week = sst_data.assign_coords(
    semana=('time', (((sst_data.time.dt.dayofyear - 1)//7) + 1).data)
)
climatologia = sst_data_with_week.groupby('semana').mean('time')

# Datos de mayo 2021
sst_surface = sst_data.sel(time=slice("2021-04-11", "2021-05-31"))
semana = (((sst_surface.time.dt.dayofyear - 1)//7) + 1).rename('semana')
sst_may_2021 = (sst_surface
                .assign_coords(semana=('time', semana.data))
                .groupby('semana')
                .mean('time'))

# Calcular anomalía
sst_anomaly_2021 = sst_may_2021 - climatologia

In [ ]:
calas_data = pd.read_csv(filename_calas)
calas_may_2021 = calas_data[calas_data['temporada'].str.contains('2021') & calas_data['semana'].between(18, 21)].copy()
calas_may_2021['lat_bin'] = pd.to_numeric(calas_may_2021['lat_bin'], errors='coerce')
calas_may_2021['lon_bin'] = pd.to_numeric(calas_may_2021['lon_bin'], errors='coerce')

boundary = gpd.read_file("/home/jupyter-daniela/suyana/geometries/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp")

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 8))
axes = axes.flatten()

semanas = [18, 19, 20, 21]

for i, semana in enumerate(semanas):
    ax = axes[i]

    if semana in sst_may_2021.semana.values:
        boundary.plot(ax=ax, facecolor='none', edgecolor='black')

        sst_week = sst_may_2021.sel(semana=semana)

        sst_plot = ax.contourf(
            sst_week.lon,
            sst_week.lat,
            sst_week['sst'],
    levels=np.linspace(13, 25, 21),
            cmap='viridis',
            vmin=13, vmax=25,
            alpha=0.8
        )
        sst_plot.set_clim(vmin=13, vmax=25)


    calas_week = calas_may_2021[calas_may_2021['semana'] == semana]
    calas_week = calas_week[calas_week['suma_pescado'] > 0]

    if not calas_week.empty:
        scatter = ax.scatter(
            calas_week['lon_bin'],
            calas_week['lat_bin'],
            c=calas_week['suma_pescado'],
            cmap='Reds',
            s=30,
            alpha=0.8,
            edgecolors='black',
            linewidth=0.3,
            vmin=0, vmax=500

        )

    ax.set_xlim([-82, -76])
    ax.set_ylim([-15, -6])
    ax.set_title(f'Semana {semana}', fontsize=10)
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')

fig.colorbar(sst_plot, ax=axes, orientation='vertical', fraction=0.025, pad=.03, label='Temperatura del Mar (°C)', shrink=0.7)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(18, 8))
axes = axes.flatten()

semanas = [18, 19, 20, 21]

for i, semana in enumerate(semanas):
    ax = axes[i]

    if semana in sst_anomaly_2021.semana.values:
        boundary.plot(ax=ax, facecolor='none', edgecolor='black')

        sst_week = sst_anomaly_2021.sel(semana=semana)

        sst_plot = ax.contourf(
            sst_week.lon,
            sst_week.lat,
            sst_week['sst'],
            levels=np.linspace(-3, 3, 21),
            cmap='RdBu_r',
            vmin=-3, vmax=3,
            alpha=0.8
        )

    calas_week = calas_may_2021[calas_may_2021['semana'] == semana]
    calas_week = calas_week[calas_week['suma_pescado'] > 0]

    if not calas_week.empty:
        scatter = ax.scatter(
            calas_week['lon_bin'],
            calas_week['lat_bin'],
            c=calas_week['suma_pescado'],
            cmap='Reds',
            s=30,
            alpha=0.8,
            edgecolors='black',
            linewidth=0.3,
            vmin=0, vmax=500
        )

    ax.set_xlim([-82, -76])
    ax.set_ylim([-15, -6])
    ax.set_title(f'Anomalía SST - Semana {semana}', fontsize=10)
    ax.set_xlabel('Longitud')
    ax.set_ylabel('Latitud')

fig.colorbar(sst_plot, ax=axes, orientation='vertical', fraction=0.025, pad=.03, label='Anomalía SST (°C)', shrink=0.7)
plt.show()